In [1]:
!pip install accelerate==0.26.0
!pip install diffusers==0.16.0
!pip install jax==0.4.23 jaxlib==0.4.23

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 7.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.8/934.8 kB 16.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 MB 21.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.26.dev20240620
    Uninstalling jaxlib-0.4.26.dev20240620:
      Successfully uninstalled jaxlib-0.4.26.dev20240620
  Attempting uninstall: jax
    Found existing installation: jax 0.4.26
    Uninstalling jax-0.4.26:
      Successfully uninstalled jax-0.4.26
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the 

In [2]:
# Clone the ImageReward repository (containing data for testing)
!git clone https://github.com/THUDM/ImageReward.git
!cd ImageReward

# Install the integrated package `image-reward`
!pip install image-reward
!pip install git+https://github.com/openai/CLIP.git

Cloning into 'ImageReward'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 224 (delta 52), reused 48 (delta 42), pack-reused 133 (from 1)
Receiving objects: 100% (224/224), 4.30 MiB | 34.92 MiB/s, done.
Resolving deltas: 100% (89/89), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 10.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 28.9 kB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 27.0 MB/s eta 0:00:00
  Created wheel for fairscale: filename=fairscale-0.4.13-py3-none-any.whl size=332106 sha256=4a536469f1837566a02762321f90896b819489f5fe1d6d85b60fc4aecd2ba81e
  Stored in directory: /root/.cache/pip/wheels/78/a4/c0/fb0a7ef03cff

In [3]:
import argparse
import copy
from dataclasses import dataclass
import functools
import json
import logging
import os
import pickle
import random

from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import ProjectConfiguration, set_seed 

import datasets
import dataclasses
from datasets import load_dataset
import diffusers
from diffusers import UNet2DConditionModel
from diffusers.loaders import AttnProcsLayers
from diffusers.models.cross_attention import LoRACrossAttnProcessor
from diffusers.optimization import get_scheduler
from diffusers.training_utils import EMAModel
from diffusers.utils import deprecate
import logging
import ImageReward as imagereward
from diffusers.utils.import_utils import is_xformers_available

import transformers
from transformers import CLIPModel, CLIPProcessor  # pylint: disable=g-multiple-import
from transformers import CLIPTextModel, CLIPTokenizer  # pylint: disable=g-multiple-import

from typing import Optional, Tuple, Union
from diffusers import DDIMScheduler
from diffusers.schedulers.scheduling_ddim import DDIMSchedulerOutput
from diffusers.utils import randn_tensor
import torch
from torch.distributions import Normal
import torch.nn.functional as F
from torch import nn
from torch.nn.parallel import DistributedDataParallel
import torch.utils.checkpoint
from tqdm.auto import tqdm

from typing import Any, Callable, Dict, List, Optional, Union
from diffusers import StableDiffusionPipeline

import datasets
from datasets import load_dataset
import numpy as np

/opt/conda/lib/python3.10/site-packages/diffusers/models/cross_attention.py:30: FutureWarning: Importing from cross_attention is deprecated. Please import from diffusers.models.attention_processor instead.
  deprecate(


In [4]:
@dataclass
class Args:
    single_prompt: str = "A green colored rabbit."
    p_batch_size: int = 4
    reward_weight: int = 10
    kl_weight: float = 0.01
    learning_rate: float = 1e-5
    adam_beta1: float = 0.9
    adam_beta2: float = 0.999
    lr_warmup_steps: int = 0
    single_flag: int = 1
    prompt_path: str = "/kaggle/input/drawbench/data_meta.json"
    gradient_accumulation_steps: int = 12
    clip_norm: float = 0.1
    g_batch_size: int = 10
    multi_gpu: int = 1
    max_train_steps: int = 1000
    v_flag: int = 1
    prompt_category: str = "all"
    p_step: int = 5
    lora_rank: int = 4
    kl_warmup: int = -1
    sft_initialization: int = 0
    reward_flag: int = 0
    reward_filter: int = 0
    v_lr: float = 1e-4
    v_batch_size: int = 24
    v_step: int = 5
    mixed_precision: str = "fp16"
    report_to: str = "tensorboard"
    output_dir: str = "online_model"
    logging_dir: str = "logs"
    seed: str = None
    pretrained_model_name_or_path: str = "runwayml/stable-diffusion-v1-5"
    revision: str = None
    prompt_category: str = "all"
    train_batch_size: int = 8
    g_step: int = 1
    gradient_checkpointing: bool = True
    adam_weight_decay: float = 0.9
    adam_epsilon: float = 1e-08
    lr_scheduler: str = "constant"
    num_train_epochs: int = 5
    buffer_size: int = 500
    num_samples: int = 1
        
@dataclasses.dataclass(frozen=False)
class TrainPolicyFuncData:
    tot_p_loss: float = 0
    tot_ratio: float = 0
    tot_kl: float = 0
    tot_grad_norm: float = 0
        
logger = get_logger(__name__, log_level="INFO")

In [5]:
class DDIMSchedulerExtended(DDIMScheduler):
    """Extension of diffusers.DDIMScheduler."""
    
    def _get_variance_logprob(self, timestep, prev_timestep):
        alpha_prod_t = self.alphas_cumprod.to(timestep.device)[timestep]
        mask_a = (prev_timestep >= 0).int().to(timestep.device)
        mask_b = 1 - mask_a
        alpha_prod_t_prev = (
            self.alphas_cumprod.to(timestep.device)[prev_timestep] * mask_a
            + self.final_alpha_cumprod.to(timestep.device) * mask_b
        )
        beta_prod_t = 1 - alpha_prod_t
        beta_prod_t_prev = 1 - alpha_prod_t_prev
        variance = (beta_prod_t_prev / beta_prod_t) * (
            1 - alpha_prod_t / alpha_prod_t_prev
        )
        return variance
    
    def step_logprob(
      self,
      model_output,
      timestep,
      sample,
      eta = 1.0,
      use_clipped_model_output = False,
      generator=None,
      variance_noise = None,
      return_dict = True):  # pylint: disable=g-bare-generic
        """Predict the sample at the previous timestep by reversing the SDE.

        Core function to propagate the diffusion process from the learned model
        outputs (most often the predicted noise).

        Args:
            model_output (`torch.FloatTensor`): direct output from learned diffusion
                  model.
            timestep (`int`): current discrete timestep in the diffusion chain.
            sample (`torch.FloatTensor`): current instance of sample being created
              by diffusion process.
            eta (`float`): weight of noise for added noise in diffusion step.
            use_clipped_model_output (`bool`): if `True`, compute "corrected"
              `model_output` from the clipped predicted original sample. Necessary
              because predicted original sample is clipped to [-1, 1] when
              `self.config.clip_sample` is `True`. If no clipping has happened,
              "corrected" `model_output` would coincide with the one provided as
              input and `use_clipped_model_output` will have not effect.
            generator: random number generator.
            variance_noise (`torch.FloatTensor`): instead of generating noise for
                  the variance using `generator`, we can directly provide the noise for
                  the variance itself. This is useful for methods such as
                  CycleDiffusion. (https://arxiv.org/abs/2210.05559)
            return_dict (`bool`): option for returning tuple rather than
                  DDIMSchedulerOutput class

        Returns:
            [`~schedulers.scheduling_utils.DDIMSchedulerOutput`] or `tuple`:
            [`~schedulers.scheduling_utils.DDIMSchedulerOutput`] if `return_dict` is
                True, otherwise a `tuple`. When
            returning a tuple, the first element is the sample tensor.

            log_prob (`torch.FloatTensor`): log probability of the sample.
        """
        if self.num_inference_steps is None:
            raise ValueError(
                  "Number of inference steps is 'None', you need to run 'set_timesteps'"
                  " after creating the scheduler"
            )

        # pylint: disable=line-too-long
        # See formulas (12) and (16) of DDIM paper https://arxiv.org/pdf/2010.02502.pdf
        # Ideally, read DDIM paper in-detail understanding

        # Notation (<variable name> -> <name in paper>
        # - pred_noise_t -> e_theta(x_t, t)
        # - pred_original_sample -> f_theta(x_t, t) or x_0
        # - std_dev_t -> sigma_t
        # - eta -> η
        # - pred_sample_direction -> "direction pointing to x_t"
        # - pred_prev_sample -> "x_t-1"

        # 1. get previous step value (=t-1)
        prev_timestep = (
            timestep - self.config.num_train_timesteps // self.num_inference_steps
        )

        # 2. compute alphas, betas
        alpha_prod_t = self.alphas_cumprod.to(timestep.device)[timestep]
        # alpha_prod_t = alpha_prod_t.to(torch.float16)
        mask_a = (prev_timestep >= 0).int().to(timestep.device)
        mask_b = 1 - mask_a
        alpha_prod_t_prev = (
            self.alphas_cumprod.to(timestep.device)[prev_timestep] * mask_a
            + self.final_alpha_cumprod.to(timestep.device) * mask_b
        )
        beta_prod_t = 1 - alpha_prod_t

        # 3. compute predicted original sample from predicted noise also called
        # "predicted x_0" of formula (12) from https://arxiv.org/pdf/2010.02502.pdf
        if self.config.prediction_type == "epsilon":
            pred_original_sample = (
                  sample - beta_prod_t ** (0.5) * model_output
          ) / alpha_prod_t ** (0.5)
        elif self.config.prediction_type == "sample":
            pred_original_sample = model_output
        elif self.config.prediction_type == "v_prediction":
            pred_original_sample = (alpha_prod_t**0.5) * sample - (
              beta_prod_t**0.5) * model_output
            # predict V
            model_output = (alpha_prod_t**0.5) * model_output + (
              beta_prod_t**0.5) * sample
        else:
            raise ValueError(
              f"prediction_type given as {self.config.prediction_type} must be one"
              " of `epsilon`, `sample`, or `v_prediction`")

        # 4. Clip "predicted x_0"
        if self.config.clip_sample:
            pred_original_sample = torch.clamp(pred_original_sample, -1, 1)

        # 5. compute variance: "sigma_t(η)" -> see formula (16)
        # σ_t = sqrt((1 − α_t−1)/(1 − α_t)) * sqrt(1 − α_t/α_t−1)
        variance = self._get_variance_logprob(timestep, prev_timestep).to(
            dtype=sample.dtype
        )
        std_dev_t = (eta * variance ** (0.5)).to(dtype=sample.dtype)

        if use_clipped_model_output:
            # the model_output is always re-derived from the clipped x_0 in Glide
            model_output = (
                sample - alpha_prod_t ** (0.5) * pred_original_sample) / beta_prod_t ** (0.5)

        # pylint: disable=line-too-long
        # 6. compute "direction pointing to x_t" of formula (12) from https://arxiv.org/pdf/2010.02502.pdf
        pred_sample_direction = (1 - alpha_prod_t_prev - std_dev_t**2) ** (0.5) * model_output

        # pylint: disable=line-too-long
        # 7. compute x_t without "random noise" of formula (12) from https://arxiv.org/pdf/2010.02502.pdf
        prev_sample = (
            alpha_prod_t_prev ** (0.5) * pred_original_sample
            + pred_sample_direction
        )

        if eta > 0:
            device = model_output.device
            if variance_noise is not None and generator is not None:
                raise ValueError(
                    "Cannot pass both generator and variance_noise. Please make sure"
                    " that either `generator` or `variance_noise` stays `None`."
                )

            if variance_noise is None:
                variance_noise = randn_tensor(
                    model_output.shape,
                    generator=generator,
                    device=device,
                    dtype=model_output.dtype)
            variance = std_dev_t * variance_noise
            dist = Normal(prev_sample, std_dev_t)
            prev_sample = prev_sample.detach().clone() + variance
            log_prob = (
                dist.log_prob(prev_sample.detach().clone())
                .mean(dim=-1)
                .mean(dim=-1)
                .mean(dim=-1)
                .detach()
                .cpu())
        if not return_dict:
            return (prev_sample,)
        
        return (
            DDIMSchedulerOutput(
                prev_sample=prev_sample, pred_original_sample=pred_original_sample
            ),
            log_prob,
        )
    
    def step_forward_logprob(
        self,
        model_output,
        timestep,
        sample,
        next_sample,
        eta = 1.0,
        use_clipped_model_output = False,
        generator=None,
        variance_noise = None,
        return_dict = True,):  # pylint: disable=g-bare-generic
        
        """Predict the sample at the previous timestep by reversing the SDE.

        Core function to propagate the diffusion process from the learned model
        outputs (most often the predicted noise).

        Args:
            model_output (`torch.FloatTensor`): direct output from learned diffusion
                model.
            timestep (`int`): current discrete timestep in the diffusion chain.
            sample (`torch.FloatTensor`): current instance of sample (x_t) being
                created by diffusion process.
            next_sample (`torch.FloatTensor`): instance of next sample (x_t-1) being
              created by diffusion process. RL sampling is the backward process,
              therefore, x_t-1 is the "next" sample of x_t.
            eta (`float`): weight of noise for added noise in diffusion step.
            use_clipped_model_output (`bool`): if `True`, compute "corrected"
                `model_output` from the clipped predicted original sample. Necessary
                because predicted original sample is clipped to [-1, 1] when
                `self.config.clip_sample` is `True`. If no clipping has happened,
                "corrected" `model_output` would coincide with the one provided as
                input and `use_clipped_model_output` will have not effect.
            generator: random number generator.
            variance_noise (`torch.FloatTensor`): instead of generating noise for
                the variance using `generator`, we can directly provide the noise for
                the variance itself. This is useful for methods such as
                CycleDiffusion. (https://arxiv.org/abs/2210.05559)
            return_dict (`bool`): option for returning tuple rather than
                DDIMSchedulerOutput class

        Returns:
        log probability.
        """
        if self.num_inference_steps is None:
            raise ValueError(
                "Number of inference steps is 'None', you need to run 'set_timesteps'"
                " after creating the scheduler"
            )

        # pylint: disable=line-too-long
        # See formulas (12) and (16) of DDIM paper https://arxiv.org/pdf/2010.02502.pdf
        # Ideally, read DDIM paper in-detail understanding

        # Notation (<variable name> -> <name in paper>
        # - pred_noise_t -> e_theta(x_t, t)
        # - pred_original_sample -> f_theta(x_t, t) or x_0
        # - std_dev_t -> sigma_t
        # - eta -> η
        # - pred_sample_direction -> "direction pointing to x_t"
        # - pred_prev_sample -> "x_t-1"

        # 1. get previous step value (=t-1)
        prev_timestep = (
            timestep - self.config.num_train_timesteps // self.num_inference_steps
        )

        # 2. compute alphas, betas
        alpha_prod_t = self.alphas_cumprod.to(timestep.device)[timestep]
        mask_a = (prev_timestep >= 0).int().to(timestep.device)
        mask_b = 1 - mask_a
        alpha_prod_t_prev = (
            self.alphas_cumprod.to(timestep.device)[prev_timestep] * mask_a
            + self.final_alpha_cumprod.to(timestep.device) * mask_b
        )
        beta_prod_t = 1 - alpha_prod_t

        # 3. compute predicted original sample from predicted noise also called
        # "predicted x_0" of formula (12) from https://arxiv.org/pdf/2010.02502.pdf
        if self.config.prediction_type == "epsilon":
            pred_original_sample = (
                sample - beta_prod_t ** (0.5) * model_output) / alpha_prod_t ** (0.5)
        elif self.config.prediction_type == "sample":
            pred_original_sample = model_output
        elif self.config.prediction_type == "v_prediction":
            pred_original_sample = (alpha_prod_t**0.5) * sample - (
              beta_prod_t**0.5) * model_output
            # predict V
            model_output = (alpha_prod_t**0.5) * model_output + (
              beta_prod_t**0.5) * sample
        else:
            raise ValueError(
              f"prediction_type given as {self.config.prediction_type} must be one"
              " of `epsilon`, `sample`, or `v_prediction`"
            )

        # 4. Clip "predicted x_0"
        if self.config.clip_sample:
            pred_original_sample = torch.clamp(pred_original_sample, -1, 1)

        # 5. compute variance: "sigma_t(η)" -> see formula (16)
        # σ_t = sqrt((1 − α_t−1)/(1 − α_t)) * sqrt(1 − α_t/α_t−1)
        variance = self._get_variance_logprob(timestep, prev_timestep).to(dtype=sample.dtype)
        std_dev_t = (eta * variance ** (0.5)).to(dtype=sample.dtype)

        if use_clipped_model_output:
            # the model_output is always re-derived from the clipped x_0 in Glide
            model_output = (
                sample - alpha_prod_t ** (0.5) * pred_original_sample) / beta_prod_t ** (0.5)

        # pylint: disable=line-too-long
        # 6. compute "direction pointing to x_t" of formula (12) from https://arxiv.org/pdf/2010.02502.pdf
        pred_sample_direction = (1 - alpha_prod_t_prev - std_dev_t**2) ** (
            0.5) * model_output

        # pylint: disable=line-too-long
        # 7. compute x_t without "random noise" of formula (12) from https://arxiv.org/pdf/2010.02502.pdf
        prev_sample = (
            alpha_prod_t_prev ** (0.5) * pred_original_sample
            + pred_sample_direction
        )

        if eta > 0:
            device = model_output.device
            if variance_noise is not None and generator is not None:
                raise ValueError(
                    "Cannot pass both generator and variance_noise. Please make sure"
                    " that either `generator` or `variance_noise` stays `None`."
                )
                
            if variance_noise is None:
                variance_noise = randn_tensor(
                    model_output.shape,
                    generator=generator,
                    device=device,
                    dtype=model_output.dtype)
            variance = std_dev_t * variance_noise
            dist = Normal(prev_sample, std_dev_t)
            log_prob = (
                dist.log_prob(next_sample.detach().clone())
                .mean(dim=-1)
                .mean(dim=-1)
                .mean(dim=-1))

        return log_prob
        

In [6]:
class StableDiffusionPipelineExtended(StableDiffusionPipeline):
    """Extension of diffusers.StableDiffusionPipeline."""

    def forward_collect_traj_ddim(
        self,
        prompt = None,
        height = None,
        width = None,
        num_inference_steps = 50,
        guidance_scale = 7.5,
        negative_prompt = None,
        num_images_per_prompt = 1,
        eta = 1.0,
        generator = None,
        latents = None,
        prompt_embeds = None,
        negative_prompt_embeds = None,
        output_type = None,
        return_dict = True,
        callback = None,
        callback_steps = 1,
        cross_attention_kwargs = None,
        is_ddp = False,
        unet_copy = None,
        soft_reward = False):
        
        # 0. Default height and width to unet
        if is_ddp:
            height = (height or self.unet.module.config.sample_size * self.vae_scale_factor)
            width = (width or self.unet.module.config.sample_size * self.vae_scale_factor)
        else:
            height = height or self.unet.config.sample_size * self.vae_scale_factor
            width = width or self.unet.config.sample_size * self.vae_scale_factor
        
        # 1. Check inputs. Raise error if not correct
        self.check_inputs(
            prompt,
            height,
            width,
            callback_steps,
            negative_prompt,
            prompt_embeds,
            negative_prompt_embeds
        )
            
        # 2. Define call parameters
        if prompt is not None and isinstance(prompt, str):
            batch_size = 1
        elif prompt is not None and isinstance(prompt, list):
            batch_size = len(prompt)
        else:
            batch_size = prompt_embeds.shape[0]
            
        device = self._execution_device
        do_classifier_free_guidance = guidance_scale > 1.0
        
        # 3. Encode input prompt
        prompt_embeds = self._encode_prompt(
            prompt,
            device,
            num_images_per_prompt,
            do_classifier_free_guidance,
            negative_prompt,
            prompt_embeds=prompt_embeds,
            negative_prompt_embeds=negative_prompt_embeds
        )
        
        # 4. Prepare timesteps
        self.scheduler.set_timesteps(num_inference_steps, device = device)
        timesteps = self.scheduler.timesteps
        
        # 5. Prepare latent variables
        if is_ddp:
            num_channels_latents = self.unet.module.in_channels
        else:
            num_channels_latents = self.unet.in_channels
        
        latents = self.prepare_latents(
            batch_size * num_images_per_prompt,
            num_channels_latents,
            height,
            width,
            prompt_embeds.dtype,
            device,
            generator,
            latents
        )
        
        latents_list = []
        log_prob_list = []
        latents_list.append(latents.detach().clone().cpu())
        
        # 6. Prepare extra step kwargs.
        extra_step_kwargs = self.prepare_extra_step_kwargs(generator, eta)
        
        # 7. Denoising loop
        num_warmup_steps = (
            len(timesteps) - num_inference_steps * self.scheduler.order)
        kl_list = []
        with self.progress_bar(total=num_inference_steps) as progress_bar:
            for i, t in enumerate(timesteps):
                # expand the latents if we are doing classifier free guidance
                latent_model_input = (
                    torch.cat([latents] * 2) if do_classifier_free_guidance else latents
                )
                
                # predict the noise residual
                noise_pred = self.unet(
                    latent_model_input,
                    t,
                    encoder_hidden_states = prompt_embeds,
                    cross_attention_kwargs = cross_attention_kwargs,
                ).sample
                
                if soft_reward:
                    old_noise_pred = unet_copy(
                        latent_model_input,
                        t,
                        encoder_hidden_states = prompt_embeds,
                        cross_attention_kwargs=cross_attention_kwargs,
                    ).sample
                    
                    if do_classifier_free_guidance:
                        old_noise_pred_uncond, old_noise_pred_text = old_noise_pred.chunk(2)
                        old_noise_pred = old_noise_pred_uncond + guidance_scale * (
                            old_noise_pred_text - old_noise_pred_uncond
                        )
                        
                # perform guidance
                if do_classifier_free_guidance:
                    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
                    noise_pred = noise_pred_uncond + guidance_scale * (
                        noise_pred_text - noise_pred_uncond
                    )
                    
                # now we get the predicted noise
                unsqueeze3x = lambda x: x[Ellipsis, None, None, None]
                unet_t = unsqueeze3x(torch.tensor([t])).to(noise_pred.device)
                # compute the previous noisy sample x_t -> x_t-1
                if soft_reward:
                    prev_latents = latents.clone()
                    
                latents, log_prob = self.scheduler.step_logprob(
                    noise_pred, unet_t, latents, **extra_step_kwargs
                )
                latents = latents.prev_sample
                latents = latents.to(prompt_embeds.dtype)
                latents_list.append(latents.detach().clone().cpu())
                log_prob_list.append(log_prob.detach().clone().cpu())
                
                if soft_reward:
                    unet_times = unsqueeze3x(torch.tensor([t] * noise_pred.shape[0])).to(
                        noise_pred.device
                    )
                    old_log_prob = (
                        self.scheduler.step_forward_logprob(
                            noise_pred,
                            unet_times,
                            prev_latents,
                            latents,
                            **extra_step_kwargs
                        ).detach().cpu()
                    )
                    
                    kl_list.append((log_prob - old_log_prob).detach().clone().cpu())
                
                # call the callback, if provided
                if i == len(timesteps) - 1 or (
                    (i + 1) > num_warmup_steps and (i + 1) % self.scheduler.order == 0):
                    progress_bar.update()
                    if callback is not None and i % callback_steps == 0:
                        callback(i, t, latents)
                        
        if output_type == "latent":
            image = latents
        elif output_type == "pil":
            # 8. Post-processing
            latents = latents.detach()
            latents = latents.to(prompt_embeds.dtype)
            image = self.decode_latents(latents)
            # 10. Convert to PIL
            image = self.numpy_to_pil(image)
        else:
            image = self.decode_latents(latents)
        
        # Offload last model to CPU
        if (
            hasattr(self, "final_offload_hook")
            and self.final_offload_hook is not None):
            self.final_offload_hook.offload()
            
        unconditional_prompt_embeds, guided_prompt_embeds = prompt_embeds.chunk(2)
        
        if soft_reward:
            kl_sum = 0
            kl_path = []
            for i in range(len(kl_list)):
                kl_sum += kl_list[i]
            kl_path.append(kl_sum.clone())
            for i in range(1, len(kl_list)):
                kl_sum -= kl_list[i - 1]
                kl_path.append(kl_sum.clone())
        else:
            kl_path = None

        return (
            image,
            latents_list,
            unconditional_prompt_embeds.detach().cpu(),
            guided_prompt_embeds.detach().cpu(),
            log_prob_list,
            kl_path,
        )
    
    # Feed transitions pairs and old model
    def forward_calculate_logprob(
        self,
        prompt_embeds,
        latents,
        next_latents,
        ts,
        unet_copy=None,
        height = None,
        width = None,
        num_inference_steps = 50,
        guidance_scale = 7.5,
        negative_prompt = None,
        num_images_per_prompt = 1,
        eta = 1.0,
        generator = None,
        negative_prompt_embeds = None,
        output_type = "pil",
        return_dict = True,
        callback = None,
        callback_steps = 1,
        cross_attention_kwargs = None,
        is_ddp = False,
        soft_reward=False):
        # pylint: disable=line-too-long
    
        if soft_reward:
            unet_copy = None
        # 0. Default height and width to unet
        if is_ddp:
            height = (
              height or self.unet.module.config.sample_size * self.vae_scale_factor
            )
            width = (
              width or self.unet.module.config.sample_size * self.vae_scale_factor
            )
        else:
            height = height or self.unet.config.sample_size * self.vae_scale_factor
            width = width or self.unet.config.sample_size * self.vae_scale_factor

        # 2. Define call parameters
        device = self._execution_device
        # here `guidance_scale` is defined analog to the guidance weight `w` of
        # equation (2) of the Imagen paper: https://arxiv.org/pdf/2205.11487.pdf .
        # `guidance_scale = 1` corresponds to doing no classifier free guidance.
        do_classifier_free_guidance = guidance_scale > 1.0

        # 3. Prepare timesteps
        self.scheduler.set_timesteps(num_inference_steps, device=device)
        timesteps = self.scheduler.timesteps
        unet_times = timesteps[ts]

        # 4. Prepare latent variables
        latents_list = []
        latents_list.append(latents.detach().clone())
        # 5. Prepare extra step kwargs.
        # TODO: Logic should ideally just be moved out of the pipeline  # pylint: disable=g-bad-todo
        extra_step_kwargs = self.prepare_extra_step_kwargs(generator, eta)

        # 6. Denoising loop
        # for training loops:
        # with self.progress_bar(total=num_inference_steps) as progress_bar:
        # for i, t in enumerate(timesteps):
        # expand the latents if we are doing classifier free guidance
        latent_model_input = (
            torch.cat([latents] * 2) if do_classifier_free_guidance else latents
        )
        time_model_input = (
            torch.cat([unet_times] * 2)
            if do_classifier_free_guidance
            else unet_times
        )

        # predict the noise residual
        noise_pred = self.unet(
            latent_model_input,
            time_model_input,
            encoder_hidden_states=prompt_embeds,
            cross_attention_kwargs=cross_attention_kwargs,
        ).sample

        # add regularization

        if unet_copy is not None:
            old_noise_pred = unet_copy(
              latent_model_input,
              time_model_input,
              encoder_hidden_states=prompt_embeds,
              cross_attention_kwargs=cross_attention_kwargs,
            ).sample

        # perform guidance
        if do_classifier_free_guidance:
            noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
            noise_pred = noise_pred_uncond + guidance_scale * (
              noise_pred_text - noise_pred_uncond
            )

        if unet_copy is not None:
            if do_classifier_free_guidance:
                old_noise_pred_uncond, old_noise_pred_text = old_noise_pred.chunk(2)
                old_noise_pred = old_noise_pred_uncond + guidance_scale * (
                    old_noise_pred_text - old_noise_pred_uncond
                )
            # now we get the predicted noise
            kl_regularizer = (noise_pred - old_noise_pred) ** 2
        else:
            kl_regularizer = torch.zeros(noise_pred.shape[0])

        unsqueeze3x = lambda x: x[Ellipsis, None, None, None]
        unet_times = unsqueeze3x(unet_times).to(noise_pred.device)
        log_prob = self.scheduler.step_forward_logprob(
            noise_pred, unet_times, latents, next_latents, **extra_step_kwargs
        )

        return log_prob, kl_regularizer

In [7]:
class ConditionalLinear(nn.Module):
    """Conditional linear."""
    
    def __init__(self, num_in, num_out, n_steps):
        super(ConditionalLinear, self).__init__()
        self.num_out = num_out
        self.lin = nn.Linear(num_in, num_out)
        self.embed = nn.Embedding(n_steps, num_out)
        self.embed.weight.data.uniform_()
        torch.nn.init.xavier_normal_(self.lin.weight)
        
    def forward(self, x, y):
        out = self.lin(x)
        gamma = self.embed(y)
        out = gamma.view(-1, self.num_out) * out
        return out
    
class ValueMulti(nn.Module):
    """ValueMulti."""
    
    def __init__(self, num_steps, img_shape):
        super(ValueMulti, self).__init__()
        self.lin1 = ConditionalLinear(int(np.prod(img_shape)) + 768, 256, num_steps)
        self.lin2 = ConditionalLinear(256, 256, num_steps)
        self.lin3 = ConditionalLinear(256, 256, num_steps)
        self.lin4 = nn.Linear(256, 1)
        torch.nn.init.xavier_normal_(self.lin4.weight)
        
    def forward(self, img, txt_emb, t):
        x = img.view(img.shape[0], -1)
        x = torch.cat([x, txt_emb], dim = 1)
        x = F.relu(self.lin1(x, t))
        x = F.relu(self.lin2(x, t))
        x = F.relu(self.lin3(x, t))
        return self.lin4(x)

def image_reward_get_reward(
    model, pil_image, prompt, weight_dtype):
    """Gets rewards using ImageReward model."""
    image = (model.preprocess(pil_image).unsqueeze(0).to(weight_dtype).to(model.device))
    image_embeds = model.blip.visual_encoder(image)
    image_atts = torch.ones(image_embeds.size()[:-1], dtype=torch.long).to(model.device)
    text_input = model.blip.tokenizer(
        prompt,
        padding = "max_length",
        truncation = True,
        max_length = 35,
        return_tensors = "pt",
    ).to(model.device)
    text_output = model.blip.text_encoder(
        text_input.input_ids,
        attention_mask = text_input.attention_mask,
        encoder_hidden_states = image_embeds,
        encoder_attention_mask = image_atts,
        return_dict = True,
    )
    txt_features = text_output.last_hidden_state[:, 0, :]
    rewards = model.mlp(txt_features)
    rewads = (rewards - model.mean) / model.std
    return rewards, txt_features

def _calculate_reward_ir(
    pipe,
    args,
    reward_tokenizer,
    tokenizer,
    weight_dtype,
    reward_clip_model,
    image_reward,
    imgs,
    prompts,
    test_flag = False
):
    """Computes reward using ImageReward model."""
    if test_flag:
        image_pil = imgs
    else:
        image_pil = pipe.numpy_to_pil(imgs)[0]
    
    blip_reward, _ = image_reward_get_reward(
        image_reward, image_pil, prompts, weight_dtype
    )
    inputs = reward_tokenizer(
        prompts,
        max_length = tokenizer.model_max_length,
        padding = "do_not_pad",
        truncation = True,
    )
    input_ids = inputs.input_ids
    padded_tokens = reward_tokenizer.pad({"input_ids": input_ids}, padding=True, return_tensors="pt")
    txt_emb = reward_clip_model.get_text_features(input_ids=padded_tokens.input_ids.to("cuda").unsqueeze(0))
    return blip_reward.cpu().squeeze(0).squeeze(0), txt_emb.squeeze(0)
    
def _collect_rollout(args, pipe, is_ddp, batch, calculate_reward, state_dict):
    """Collects trajectories."""
    for _ in range(args.g_step):
        with torch.no_grad():
            (
                image,
                latents_list,
                unconditional_prompt_embeds,
                guided_prompt_embeds,
                log_prob_list,
                _,
            ) = pipe.forward_collect_traj_ddim(prompt=batch, is_ddp = is_ddp)
            reward_list = []
            txt_emb_list = []
            for i in range(len(batch)):
                reward, txt_emb = calculate_reward(image[i], batch[i])
                reward_list.append(reward)
                txt_emb_list.append(txt_emb)
            reward_list = torch.stack(reward_list).detach().cpu()
            txt_emb_list = torch.stack(txt_emb_list).detach().cpu()
            # store the roll out data
            for i in range(len(latents_list) - 1):
                # deal with a batch of data in each step i
                state_dict["prompt"].extend(batch)
                state_dict["state"] = torch.cat((state_dict["state"], latents_list[i]))
                state_dict["next_state"] = torch.cat(
                    (state_dict["next_state"], latents_list[i + 1])
                )
                state_dict["timestep"] = torch.cat(
                    (state_dict["timestep"], torch.LongTensor([i] * len(batch)))
                )
                state_dict["final_reward"] = torch.cat(
                    (state_dict["final_reward"], reward_list)
                )
                state_dict["unconditional_prompt_embeds"] = torch.cat((
                    state_dict["unconditional_prompt_embeds"],
                    unconditional_prompt_embeds,
                ))
                state_dict["guided_prompt_embeds"] = torch.cat(
                    (state_dict["guided_prompt_embeds"], guided_prompt_embeds)
                )
                state_dict["txt_emb"] = torch.cat((state_dict["txt_emb"], txt_emb_list))
                state_dict["log_prob"] = torch.cat(
                    (state_dict["log_prob"], log_prob_list[i])
                )
            del (
                image,
                latents_list,
                unconditional_prompt_embeds,
                guided_prompt_embeds,
                reward_list,
                txt_emb_list,
                log_prob_list,
                reward,
                txt_emb,
            )
            torch.cuda.empty_cache()
            
def _trim_buffer(buffer_size, state_dict):
    """Delete old samples from the bufffer."""
    if state_dict["state"].shape[0] > buffer_size:
        state_dict["prompt"] = state_dict["prompt"][-buffer_size:]
        state_dict["state"] = state_dict["state"][-buffer_size:]
        state_dict["next_state"] = state_dict["next_state"][-buffer_size:]
        state_dict["timestep"] = state_dict["timstep"][-buffer_size:]
        state_dict["final_reward"] = state_dict["final_reward"][-buffer_size:]
        state_dict["unconditional_prompt_embeds"] = state_dict[
            "unconditional_prompt_embeds"
        ][-buffer_size:]
        state_dict["guided_prompt_embeds"] = state_dict["guided_prompt_embeds"][
            -buffer_size:
        ]
        state_dict["txt_emb"] = state_dict["txt_emb"][-buffer_size:]
        state_dict["log_prob"] = state_dict["log_prob"][-buffer_size:]
        
def _get_batch(data_iter_loader, data_iterator, prompt_list, args, accelerator):
    """creates a batch"""
    batch = next(data_iter_loader, None)
    if batch is None:
        batch = next(
            iter(
                accelerator.prepare(
                    data_iterator(prompt_list, batch_size = args.g_batch_size)
                )
            )
        )
    if args.single_flag == 1:
        for i in range(len(batch)):
            batch[i] = args.single_prompt
    batch_list = []
    for i in range(len(batch)):
        batch_list.extend([batch[i] for _ in range(args.num_samples)])
    batch = batch_list
    return batch
    
        
def _save_model(args, count, is_ddp, accelerator, unet):
    """Saves UNET model."""
    save_path = os.path.join(args.output_dir, f"save_{count}")
    print(f"Saving model to {save_path}")
    if is_ddp:
        unet_to_save = copy.deepcopy(accelerator.unwrap_model(unet)).to(torch.float32)
        unet_to_save.save_attn_procs(save_path)
    else:
        unet_to_save = copy.deepcopy(unet).to(torch.float32)
        unet_to_save.save_attn_procs(save_path)
        
def get_random_indices(num_indices, sample_size):
    """Returns a random sample of indices from a larger list of indices."""
    if num_indices < sample_size:
        sample_size = num_indices
    return np.random.choice(num_indices, size=sample_size, replace = False)

def _train_value_func(value_function, state_dict, accelerator, args):
    """Trains the value function."""
    indices = get_random_indices(state_dict["state"].shape[0], args.v_batch_size)
    batch_state = state_dict["state"][indices]
    batch_timestep = state_dict["timestep"][indices]
    batch_final_reward = state_dict["final_reward"][indices]
    batch_txt_emb = state_dict["txt_emb"][indices]
    pred_value = value_function(
        batch_state.cuda().detach(),
        batch_txt_emb.cuda().detach(),
        batch_timestep.cuda().detach()
    )
    batch_final_reward = batch_final_reward.cuda().float()
    value_loss = F.mse_loss(
      pred_value.float().reshape([args.v_batch_size, 1]),
      batch_final_reward.cuda().detach().reshape([args.v_batch_size, 1]))
    accelerator.backward(value_loss / args.v_step)
    del (
        pred_value,
        batch_state,
        batch_timestep,
        batch_final_reward,
        batch_txt_emb
    )
    return (value_loss.item() / args.v_step)

def _train_policy_func(
    args,
    state_dict,
    pipe,
    unet_copy,
    is_ddp,
    count,
    policy_steps,
    accelerator,
    tpfdata,
    value_function
):
    """Trains the policy function."""
    with torch.no_grad():
        indices = get_random_indices(state_dict["state"].shape[0], args.p_batch_size)
        batch_state = state_dict["state"][indices]
        batch_next_state = state_dict["next_state"][indices]
        batch_timestep = state_dict["timestep"][indices]
        batch_final_reward = state_dict["final_reward"][indices]
        batch_unconditional_prompt_embeds = state_dict["unconditional_prompt_embeds"][indices]
        batch_guided_prompt_embeds = state_dict["guided_prompt_embeds"][indices]
        batch_prompt_embeds = torch.cat(
            [batch_unconditional_prompt_embeds, batch_guided_prompt_embeds]
        )
        batch_txt_emb = state_dict["txt_emb"][indices]
        batch_log_prob = state_dict["log_prob"][indices]
        
    # calculate loss from the custom function
    log_prob, kl_regularizer = pipe.forward_calculate_logprob(
        prompt_embeds = batch_prompt_embeds.cuda(),
        latents = batch_state.cuda(),
        next_latents = batch_next_state.cuda(),
        ts = batch_timestep.cuda(),
        unet_copy = unet_copy,
        is_ddp = is_ddp,
    )
    with torch.no_grad():
        adv = batch_final_reward.cuda().reshape([args.p_batch_size, 1]) - value_function(
            batch_state.cuda(),
            batch_txt_emb.cuda(),
            batch_timestep.cuda()
        ).reshape([args.p_batch_size, 1])
    ratio = torch.exp(log_prob - batch_log_prob.cuda())
    ratio = torch.clamp(ratio, 1.0 - args.ratio_clip, 1.0 + args.ratio_clip)
    loss = (
        -args.reward_weight
        * adv.detach().float()
        * ratio.float().reshape([args.p_batch_size, 1])
    ).mean()
    if count > args.kl_warmup:
        loss += args.kl_weight * kl_regularizer.mean()
    loss = loss / (args.gradient_accumulation_steps)
    accelerator.backward(loss)
    # logging
    tpfdata.tot_ratio += ratio.mean().item() / policy_steps
    tpfdata.tot_kl += kl_regularizer.mean().item() / policy_steps
    tpfdata.tot_p_loss += loss.item() / policy_steps
    

In [8]:
def _update_output_dir(args):
    """Modifies `args.output_dir` using configurations in `args`.
    
    Args:
        args: argparse.Namespace object.
    """
    if args.single_flag == 1:
        data_log = "single_prompt/" + args.single_prompt.replace(" ", "_") + "/"
    else:
        data_log = args.prompt_path.split("/")[-2] + "_"
        data_log += args.prompt_category 
    learning_log = "p_lr" + str(args.learning_rate) + "_s" + str(args.p_step)
    learning_log += (
        "_b"
        + str(args.p_batch_size)
        + "_g"
        + str(args.gradient_accumulation_steps)
    )
    learning_log += "_l" + str(args.lora_rank)
    coeff_log = "_kl" + str(args.kl_weight) + "_re" + str(args.reward_weight)
    if args.kl_warmup > 0:
        coeff_log += "_klw" + str(args.kl_warmup)
    if args.sft_initialization == 0:
        start_log = "pre_train/"
    else:
        start_log = "/sft/"
    if args.reward_flag == 0:
        args.output_dir += "/img_reward_{}/".format(args.reward_filter)
    else:
        args.output_dir += "/pregreward_{}/".format(args.reward_filter)
    args.output_dir += start_log + data_log + "/" + learning_log + coeff_log
    if args.v_flag == 1:
        value_log = "_v_lr" + str(args.v_lr) + "_b" + str(args.v_batch_size)
        value_log += "_s" + str(args.v_step)
        args.output_dir += value_log
    return args

In [9]:
def train_model(orig_args):
    # Update output paths for logging purposes.
    args = _update_output_dir(orig_args)
    logging_dir = os.path.join(args.output_dir, args.logging_dir)
    accelerator_project_config = ProjectConfiguration(
        logging_dir = logging_dir, total_limit = 2
    )
    accelerator = Accelerator(
        mixed_precision = args.mixed_precision,
        log_with = args.report_to,
        project_config = accelerator_project_config
    )
    # Make one log on every process with the configuration for debugging.
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    logger.info(accelerator.state, main_process_only=False)
    if accelerator.is_local_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_warning()
        diffusers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()
        diffusers.utils.logging.set_verbosity_error()
    
    # Handle the repository creation
    if accelerator.is_main_process:
        os.makedirs(args.output_dir, exist_ok=True)
    # Load scheduler, tokenizer and models.
    tokenizer = CLIPTokenizer.from_pretrained(
        args.pretrained_model_name_or_path,
        subfolder="tokenizer",
        revision=args.revision
    )
    text_encoder = CLIPTextModel.from_pretrained(
        args.pretrained_model_name_or_path,
        subfolder="text_encoder",
        revision=args.revision,
    )
    weight_dtype = torch.float16
    
    # reward models
    reward_clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
    reward_processor = CLIPProcessor.from_pretrained(
        "openai/clip-vit-large-patch14"
    )
    reward_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
    
    if args.reward_flag == 0:
        image_reward = imagereward.load("ImageReward-v1.0")
    else:
        reward_model = pickle.load(open(args.reward_model_path, "rb"))["reward"]
    image_reward.requires_grad_(False)
    image_reward.to(accelerator.device, dtype=weight_dtype)
    reward_clip_model.requires_grad_(False)
    
    pipe = StableDiffusionPipelineExtended.from_pretrained(
        args.pretrained_model_name_or_path, torch_dtype=weight_dtype
    )
    unet = UNet2DConditionModel.from_pretrained(
        args.pretrained_model_name_or_path,
        subfolder = "unet",
        revision = args.revision
    )
    pipe.scheduler = DDIMSchedulerExtended.from_config(pipe.scheduler.config)
    vae = pipe.vae
    unet.requires_grad_(False)
    unet.eval()
    text_encoder = pipe.text_encoder
    
    # Freeze vae and text_encoder
    vae.requires_grad_(False)
    text_encoder.requires_grad_(False)
    # pretrain model to calculate kl
    unet_copy = UNet2DConditionModel.from_pretrained(
        args.pretrained_model_name_or_path,
        subfolder="unet",
        revision=args.revision,
    )
    # freeze unet copy
    unet_copy.requires_grad_(False)
    # Move text_encode and vae to gpu and cast to weight_dtype
    text_encoder.to(accelerator.device, dtype=weight_dtype)
    vae.to(accelerator.device, dtype=weight_dtype)
    unet.to(accelerator.device, dtype=weight_dtype)
    unet_copy.to(accelerator.device, dtype=weight_dtype)
    
    # Define lora layers
    lora_attn_procs = {}
    for name in unet.attn_processors.keys():
        cross_attention_dim = (
            None
            if name.endswith("attn1.processor")
            else unet.config.cross_attention_dim
        )
        if name.startswith("mid_block"):
            hidden_size = unet.config.block_out_channels[-1]
        elif name.startswith("up_block"):
            block_id =  int(name[len("up_blocks.")])
            hidden_size = list(reversed(unet.config.block_out_channels))[block_id]
        elif name.startswith("down_block"):
            block_id = int(name[len("down_blocks.")])
            hidden_size = unet.config.block_out_channels[block_id]
        
        lora_attn_procs[name] = LoRACrossAttnProcessor(
            hidden_size = hidden_size,
            cross_attention_dim = cross_attention_dim,
            rank = args.lora_rank
        )
        
    unet.set_attn_processor(lora_attn_procs)
    lora_layers = AttnProcsLayers(unet.attn_processors)
    
    if args.gradient_checkpointing:
        unet.enable_gradient_checkpointing()
    
    unet.enable_gradient_checkpointing()
    
    # Initialize the optimizer
    optimizer_cls = torch.optim.AdamW
    
    optimizer = optimizer_cls(
        lora_layers.parameters(),
        lr = args.learning_rate,
        betas = (args.adam_beta1, args.adam_beta2),
        weight_decay = args.adam_weight_decay,
        eps = args.adam_epsilon,
    )
    
    with open(args.prompt_path) as json_file:
        prompt_dict = json.load(json_file)
    
    prompt_list = []
    for prompt in prompt_dict:
        category = prompt_dict[prompt]["category"]
        prompt_list.append(prompt)
    
    # Data iterator for prompt dataset
    def _my_data_iterator(data, batch_size):
        # Shuffle the data randomly
        random.shuffle(data)
        
        for i in range(0, len(data), batch_size):
            batch = data[i : i + batch_size]
            yield batch
    
    data_iterator = _my_data_iterator(prompt_list, batch_size = args.g_batch_size)
    data_iterator = accelerator.prepare(data_iterator)
    
    lr_scheduler = get_scheduler(
        args.lr_scheduler,
        optimizer = optimizer,
        num_warmup_steps = args.lr_warmup_steps * args.gradient_accumulation_steps,
        num_training_steps = args.max_train_steps * args.gradient_accumulation_steps,
    )
    
    value_function = ValueMulti(50, (4, 64, 64))
    value_optimizer = torch.optim.AdamW(value_function.parameters(), lr = args.v_lr)
    value_function, value_optimizer = accelerator.prepare(value_function, value_optimizer)
    
    lora_layers, optimizer, lr_scheduler = accelerator.prepare(lora_layers, optimizer, lr_scheduler)
    
    reward_clip_model.to(accelerator.device, dtype=weight_dtype)
    
    if accelerator.is_main_process:
        accelerator.init_trackers("text2image-fine-tune", config = vars(args))
        
    # Train!
    total_batch_size = (
        args.train_batch_size
        * accelerator.num_processes
        * args.gradient_accumulation_steps
    )
    global_step = 0
    
    logger.info("***** Running training *****")
    logger.info(f"  Num Epochs = {args.num_train_epochs}")
    logger.info(
      f"  Instantaneous batch size per device = {args.train_batch_size}")
    logger.info(
      "  Total train batch size (w. parallel, distributed & accumulation) ="
      f" {total_batch_size}")
    logger.info(
      f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
    logger.info(
      f"  Total optimization steps = {args.max_train_steps // args.p_step}")
    
    pipe.scheduler = DDIMSchedulerExtended.from_config(pipe.scheduler.config)
    pipe.to("cuda")
    
    # Only show the progress bar once on each machine.
    progress_bar = tqdm(
        range(global_step, args.max_train_steps),
        disable=not accelerator.is_local_main_process,
    )
    progress_bar.set_description("Steps")

    def _map_cpu(x):
        return x.cpu()
    
    state_dict = {}
    state_dict["prompt"] = []
    state_dict["state"] = _map_cpu(torch.FloatTensor().to(weight_dtype))
    state_dict["next_state"] = _map_cpu(torch.FloatTensor().to(weight_dtype))
    state_dict["timestep"] = _map_cpu(torch.LongTensor())
    state_dict["final_reward"] = _map_cpu(torch.FloatTensor().to(weight_dtype))
    state_dict["unconditional_prompt_embeds"] = _map_cpu(
          torch.FloatTensor().to(weight_dtype)
    )
    state_dict["guided_prompt_embeds"] = _map_cpu(
        torch.FloatTensor().to(weight_dtype)
    )
    state_dict["txt_emb"] = _map_cpu(torch.FloatTensor().to(weight_dtype))
    state_dict["log_prob"] = _map_cpu(torch.FloatTensor().to(weight_dtype))
    
    calculate_reward = functools.partial(
        _calculate_reward_ir,
        pipe,
        args,
        reward_tokenizer,
        tokenizer,
        weight_dtype,
        reward_clip_model,
        image_reward
    )
    
    count = 0
    buffer_size = args.buffer_size
    policy_steps = args.gradient_accumulation_steps * args.p_step
    data_iter_loader = iter(data_iterator)
    is_ddp = isinstance(unet, DistributedDataParallel)
    pipe.unet = unet
    print("model is parallel:", is_ddp)
    
    for count in range(0, args.max_train_steps // args.p_step):
        # fix batchnorm
        unet.eval()
        
        batch = _get_batch(
            data_iter_loader, _my_data_iterator, prompt_list, args, accelerator
        )
        _collect_rollout(args, pipe, is_ddp, batch, calculate_reward, state_dict)
        _trim_buffer(buffer_size, state_dict)
        
        total_val_loss = 0
        value_optimizer.zero_grad()
        for v_step in range(args.v_step):
            if v_step < args.v_step - 1:
                with accelerator.no_sync(value_function):
                    total_val_loss += _train_value_func(
                        value_function, state_dict, accelerator, args
                    )
            else:
                total_val_loss += _train_value_func(
                    value_function, state_dict, accelerator, args
                )
        value_optimizer.step()
        value_optimizer.zero_grad()
        if accelerator.is_local_main_process:
            print("value_loss", total_val_loss)
            accelerator.log({"value_loss": total_val_loss}, step=count)
        del total_val_loss
        torch.cuda.empty_cache()
        
        # policy learning
        tpfdata = TrainPolicyFuncData()
        for _ in range(args.p_step):
            optimizer.zero_grad()
            for accum_step in range(int(args.gradient_accumulation_steps)):
                if accum_step < int(args.gradient_accumulation_steps) - 1:
                    with accelerator.no_sync(unet):
                        _train_policy_func(
                            args,
                            state_dict,
                            pipe,
                            unet_copy,
                            is_ddp,
                            count,
                            policy_steps,
                            accelerator,
                            tpfdata,
                            value_function
                        )
                else:
                    _train_policy_func(
                        args,
                        state_dict,
                        pipe,
                        unet_copy,
                        is_ddp,
                        count,
                        policy_steps,
                        accelerator,
                        tpfdata,
                        value_function
                    )
            
            if accelerator.sync_gradients:
                norm = accelerator.clip_grad_norm_(unet.parameters(), args.clip_norm)
            tpfdata.tot_grad_norm += norm.item() / args.p_step
            optimizer.step()
            lr_scheduler.step()
            if accelerator.is_main_process:
                print(f"count: [{count} / {args.max_train_steps // args.p_step}]")
                print("train_reward", torch.mean(state_dict["final_reward"]).item())
                accelerator.log(
                    {"train_reward": torch.mean(state_dict["final_reward"]).item()},
                    step=count,
                )
                print("grad norm", tpfdata.tot_grad_norm, "ratio", tpfdata.tot_ratio)
                print("kl", tpfdata.tot_kl, "p_loss", tpfdata.tot_p_loss)
                accelerator.log({"grad norm": tpfdata.tot_grad_norm}, step=count)
                accelerator.log({"ratio": tpfdata.tot_ratio}, step=count)
                accelerator.log({"kl": tpfdata.tot_kl}, step=count)
                accelerator.log({"p_loss": tpfdata.tot_p_loss}, step=count)
            torch.cuda.empty_cache()
            
        if accelerator.sync_gradients:
            global_step += 1
            if global_step % args.checkpointing_steps == 0:
                if accelerator.is_main_process:
                    save_path = os.path.join(args.output_dir, f"checkpoint-{global_step}")
                    accelerator.save_state(output_dir=save_path)
                    logger.info(f"Saved state to {save_path}")
            print("global_step", global_step)
        
        # save model per interval
        if count % args.save_interval == 0:
            accelerator.wait_for_everyone()
            if accelerator.is_main_process:
                _save_model(args, count, is_ddp, accelerator, unet)
    
    accelerator.wait_for_everyone()
    if accelerator.is_main_process:
        _save_model(args, count, is_ddp, accelerator, unet)
    
    accelerator.end_training()

In [10]:
args = Args()
train_model(args)

tokenizer/tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

...

{'requires_safety_checker'} was not found in config. Values will be initialized to default values.
{'scaling_factor'} was not found in config. Values will be initialized to default values.
{'prediction_type'} was not found in config. Values will be initialized to default values.
{'time_embedding_act_fn', 'num_class_embeds', 'upcast_attention', 'resnet_time_scale_shift', 'conv_in_kernel', 'mid_block_type', 'time_embedding_type', 'only_cross_attention', 'projection_class_embeddings_input_dim', 'use_linear_projection', 'timestep_post_act', 'encoder_hid_dim', 'resnet_out_scale_factor', 'class_embed_type', 'time_embedding_dim', 'class_embeddings_concat', 'resnet_skip_time_act', 'cross_attention_norm', 'dual_cross_attention', 'addition_embed_type_num_heads', 'time_cond_proj_dim', 'conv_out_kernel', 'mid_block_only_cross_attention', 'addition_embed_type'} was not found in config. Values will be initialized to default values.
{'time_embedding_act_fn', 'num_class_embeds', 'upcast_attention', 'r

  0%|          | 0/1000 [00:00<?, ?it/s]

model is parallel: False


  0%|          | 0/50 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.00 GiB. GPU 0 has a total capacity of 15.89 GiB of which 4.11 GiB is free. Process 2946 has 11.77 GiB memory in use. Of the allocated memory 11.23 GiB is allocated by PyTorch, and 255.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)